# SARIMAX forecasting on fixed consumed

#### Import data

In [22]:
import warnings
warnings.filterwarnings("ignore")
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sktime.forecasting.arima import ARIMA

import DataRetriever as dr

retriever = dr.DataRetriever()

year2 = retriever.get_data("All-Subsystems-hour-Year2.pkl")

all_consuming = retriever.get_attributes(file_name='consuming_attributes.pkl')
flex_consuming = ["Load_ClothesWasherPowerWithStandby", "Elec_PowerDishwasher", "Load_DryerPowerTotal"]
fixed_consuming = list(set(all_consuming) - set(flex_consuming))

con_df = pd.DataFrame(retriever.get_data(
    file_name='All-Subsystems-hour-Year2.pkl')[fixed_consuming].sum(axis=1).clip(lower=0) / 1e3
                      ) #Convert til kWh
con_df.columns = ['Fixed Consumed']

#con_df = con_df.resample('4h').sum()

con_df

,Fixed Consumed
Timestamp,
2015-02-01 00:00:00,1.748570
2015-02-01 01:00:00,2.216490
2015-02-01 02:00:00,1.941349
2015-02-01 03:00:00,1.750880
2015-02-01 04:00:00,1.979749
...,...
2016-01-31 19:00:00,1.016179
2016-01-31 20:00:00,0.647665
2016-01-31 21:00:00,0.962249


In [23]:
fig = go.Figure(go.Scattergl(
    x = con_df.index,
    y = con_df["Fixed Consumed"]
))

fig.update_yaxes(title='Fixed Consumed Energy [kWh]')

fig.show()

In [24]:
from statsmodels.tsa.stattools import adfuller

ADF_test = adfuller(con_df["Fixed Consumed"])

print(f"The p-value from the Augmented Dickey-Fuller test is {ADF_test[1]}.")

The p-value from the Augmented Dickey-Fuller test is 1.5944098012795607e-09.


In [25]:
# Calculate the seasonal difference as a weekly seasonality
con_df["Seasonal Difference"] = con_df["Fixed Consumed"] - con_df["Fixed Consumed"].shift(168)

In [26]:
fig = go.Figure(go.Scattergl(
    x = con_df.index,
    y = con_df["Seasonal Difference"]
))

fig.update_yaxes(title='Fixed Consumed Energy [kWh]')

fig.show()

In [27]:
con_df['Seasonal 1dif'] = con_df['Seasonal Difference'] - con_df['Seasonal Difference'].shift(1)

In [28]:
fig = go.Figure(go.Scattergl(
    x = con_df.index,
    y = con_df["Seasonal 1dif"]
))

fig.update_yaxes(title='Fixed Consumed Energy [kWh]')

fig.show()

#### Train, Validation and Test data

In [29]:
#Doing the training, validation and test splits now so they can be used for forecasting later.

In [30]:
year2

,Timestamp,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,...,SHW_GlycolFlowHXCoriolisSHW,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,2015-02-01 00:00:00,0.060105,4.957915,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.449170,1045.817746,4.498593,72.228102,74.959034,0.0
2015-02-01 01:00:00,2015-02-01 01:00:00,0.131439,4.904702,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,391.606267,1042.026917,4.442783,71.963450,73.979483,0.0
2015-02-01 02:00:00,2015-02-01 02:00:00,0.194187,5.054887,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,388.484683,1023.145217,4.444083,71.921017,73.818717,0.0
2015-02-01 03:00:00,2015-02-01 03:00:00,0.258256,4.991525,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.792833,1066.177350,4.467983,71.844683,74.535050,0.0
2015-02-01 04:00:00,2015-02-01 04:00:00,0.322324,5.040868,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,371.120983,1081.112783,4.466550,71.959983,74.741467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-31 19:00:00,2016-01-31 19:00:00,1.472255,4.810046,0.0,0.750000,0.750000,0.750000,0.766667,0.750000,0.0,...,419.930636,942.630328,0.001339,0.000336,11.229733,22.489150,4.243500,67.764183,68.643883,0.0
2016-01-31 20:00:00,2016-01-31 20:00:00,1.546231,4.917617,0.0,0.750000,0.750000,0.750000,0.483333,1.000000,0.0,...,419.930636,942.630328,0.001503,0.000385,11.366567,22.483300,4.275667,67.889300,68.772900,0.0
2016-01-31 21:00:00,2016-01-31 21:00:00,1.617565,4.820712,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.0,...,419.930636,942.630328,0.001470,0.000437,11.448617,22.509617,4.297500,67.785950,68.651417,0.0


In [31]:
exog_attributes = pd.DataFrame(year2['OutEnv_OutdoorAmbTemp']) # If we want exogenous attributes in the model as well.
#exog_attributes = exog_attributes[0:8000]

In [32]:
X = pd.merge(con_df, exog_attributes, how='left', on=con_df.index)
X.drop(columns=['key_0'], inplace=True)
X.index = con_df.index
#X.drop(columns=['1dif', '2dif', '3dif', 'Seasonal Difference'], inplace=True)
X.rename(columns={'OutEnv_OutdoorAmbTemp': 'Outdoor Temp'},inplace=True)
#X = con_df
X = X['2015-02-01 00:00:00' : '2016-01-23 00:00:00']

In [33]:
prediction_range = 24 * 3  #Three days

In [34]:
test_df = X[len(X) - prediction_range : ] #Last 3 days of observations
test_df

,Fixed Consumed,Seasonal Difference,Seasonal 1dif,Outdoor Temp
Timestamp,,,,
2016-01-20 01:00:00,2.524546,0.669950,0.428337,-7.848062
2016-01-20 02:00:00,2.238024,0.646698,-0.023253,-8.004554
2016-01-20 03:00:00,2.422030,0.596842,-0.049856,-8.131746
2016-01-20 04:00:00,2.482557,0.792985,0.196143,-8.328567
2016-01-20 05:00:00,2.213323,0.522031,-0.270954,-8.560276
...,...,...,...,...
2016-01-22 20:00:00,3.404545,2.577446,0.869212,-6.847771
2016-01-22 21:00:00,3.617934,2.451614,-0.125832,-6.889480
2016-01-22 22:00:00,2.568728,1.526304,-0.925310,-6.760921


In [35]:
X.drop(test_df.index, inplace=True)

validation_df = X[len(X) - prediction_range : ] #Last 3 days of observations after test_df indexes have been dropped
validation_df

,Fixed Consumed,Seasonal Difference,Seasonal 1dif,Outdoor Temp
Timestamp,,,,
2016-01-17 01:00:00,1.294820,0.438007,0.240268,2.627829
2016-01-17 02:00:00,1.195170,0.625398,0.187391,2.371032
2016-01-17 03:00:00,1.159780,0.406732,-0.218667,2.538034
2016-01-17 04:00:00,1.157602,0.413899,0.007168,2.319922
2016-01-17 05:00:00,1.153567,0.583117,0.169218,2.009884
...,...,...,...,...
2016-01-19 20:00:00,2.430027,1.019222,0.064696,-6.915420
2016-01-19 21:00:00,1.992742,-0.312478,-1.331700,-7.262263
2016-01-19 22:00:00,2.487860,-0.222584,0.089894,-7.502328


In [36]:
train_df = X[ : len(X)-prediction_range]
train_df_1week = train_df.index[-1 - (24*7 + 5)] #Used later for plots
train_df_1week, train_df

(Timestamp('2016-01-09 19:00:00', freq='H'),
                      Fixed Consumed  Seasonal Difference  Seasonal 1dif  \
 Timestamp                                                                 
 2015-02-01 00:00:00        1.748570                  NaN            NaN   
 2015-02-01 01:00:00        2.216490                  NaN            NaN   
 2015-02-01 02:00:00        1.941349                  NaN            NaN   
 2015-02-01 03:00:00        1.750880                  NaN            NaN   
 2015-02-01 04:00:00        1.979749                  NaN            NaN   
 ...                             ...                  ...            ...   
 2016-01-16 20:00:00        1.513843            -0.197341       0.116459   
 2016-01-16 21:00:00        1.195411             0.271280       0.468622   
 2016-01-16 22:00:00        1.559334             0.391973       0.120693   
 2016-01-16 23:00:00        1.129336             0.174051      -0.217922   
 2016-01-17 00:00:00        1.583407       

#### Determine order of autoregressive terms (p) and moving average terms (q)

In [37]:
#Because of observations from ACF for original data and seasonal diffed data, we use the seasonal 1-order diffed data.

acf_values, acf_conf = acf(con_df['Seasonal 1dif'].dropna(), nlags=169, alpha=0.05)

for i in range(len(acf_values)):
    acf_conf[i] = acf_conf[i] - acf_values[i]

acf_conf = pd.DataFrame(acf_conf, columns=['Upper', 'Lower'])

fig = go.Figure(go.Bar(
    y = acf_values
))

fig.add_trace(go.Scatter(
    x = list(acf_conf.index) + list(acf_conf.index[::-1]),
    y = list(acf_conf['Upper']) + list(acf_conf['Lower'][::-1]),
    fill='toself',
    fillcolor='rgba(84,84,84,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_yaxes(title="Autocorrelation", range=[-1, 1])
fig.update_xaxes(title="Lag")

fig.show()

In [38]:
pacf_values, pacf_conf = pacf(con_df['Seasonal 1dif'].dropna(), nlags=169, alpha=0.05)

for i in range(len(pacf_values)):
    pacf_conf[i] = pacf_conf[i] - pacf_values[i]

pacf_conf = pd.DataFrame(pacf_conf, columns=['Upper', 'Lower'])

fig = go.Figure(go.Bar(
    y = pacf_values
))

fig.add_trace(go.Scatter(
    x = list(pacf_conf.index) + list(pacf_conf.index[::-1]),
    y = list(acf_conf['Upper']) + list(acf_conf['Lower'][::-1]),
    fill='toself',
    fillcolor='rgba(84,84,84,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_yaxes(title="Partial Autocorrelation", range=[-1, 1])
fig.update_xaxes(title="Lag")

fig.show()

    #### SARIMA

In [39]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def arimamodel(*, training_data, trend='c', p, d, q, P=0, D=0, Q=0, m=0):

    model = SARIMAX(training_data, trend=trend, order=(p, d, q), seasonal_order=(P, D, Q, m))
    fitted_model = model.fit(maxiter=15)

    return fitted_model

def arima_forecast(data, fitted_model, *, start_index, end_index):

    data['Prediction'] = fitted_model.predict(start = start_index, end = end_index)

    predictions = fitted_model.get_forecast(end_index)
    #yhat = test.predicted_mean
    yhat_conf = predictions.conf_int(alpha=0.9)

    return data, yhat_conf


def plot_forecast(data, conf_interval, bands=True):

    fig = go.Figure(go.Scattergl(
        x = data.index,
        y = data['Generated Energy'],
        name = "Observed Values"
    ))

    fig.add_trace(go.Scattergl(
        x = data.index,
        y = data['Prediction'],
        name = "Predicted Values"
    ))

    if bands == True:
        fig.add_trace(go.Scattergl(
            x = conf_interval.index,
            y = conf_interval['lower Generated Energy'],
            name = "Lower Bound"
        ))

        fig.add_trace(go.Scattergl(
            x = conf_interval.index,
            y = conf_interval['upper Generated Energy'],
            name = "Upper Bound"
        ))

    fig.show()

In [40]:
# con_df_resampled = con_df.resample('4h').sum()
# con_df_resampled

In [41]:
# train_df_resampled = train_df.resample('4h').sum()
# validation_df_resampled = validation_df.resample('4h').sum()
# test_df_resampled = test_df.resample('4h').sum()

In [ ]:
model1 = arimamodel(training_data=train_df['Fixed Consumed'], trend='c', p=4, d=1, q=2, P=1, D=1, Q=0, m=168)
model1.summary()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.19856D-01    |proj g|=  7.50044D-01


 This problem is unconstrained.


In [ ]:
forecast1, conf_int1 = arima_forecast(data=con_df, fitted_model=model1,
                                      start_index=0, end_index=validation_df.index[-1])
forecast1

In [ ]:
from sklearn.metrics import mean_squared_error

y_true = forecast1['Fixed Consumed'][validation_df.index[0] : validation_df.index[-1]]
y_pred = forecast1['Prediction'][validation_df.index[0] : validation_df.index[-1]]

model1_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
print(model1_rmse)
model1_rmse

In [ ]:
fig = go.Figure(go.Scattergl(
    x = forecast1[train_df_1week : validation_df.index[0]].index,
    y = forecast1['Fixed Consumed'][train_df_1week : validation_df.index[0]],
    name = "Training Data",
    line=dict(color='rgb(84, 84, 84)')
))

fig.add_trace(go.Scattergl(
    x = forecast1[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast1['Fixed Consumed'][validation_df.index[0] : validation_df.index[-1]],
    name = "Observed",
    line=dict(color='rgb(234,143,129)')
))

fig.add_trace(go.Scattergl(
    x = forecast1[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast1['Prediction'][validation_df.index[0] : validation_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

# if bands == True:
#     fig.add_trace(go.Scattergl(
#         x = conf_interval.index,
#         y = conf_interval['lower Generated Energy'],
#         name = "Lower Bound"
#     ))
#
#     fig.add_trace(go.Scattergl(
#         x = conf_interval.index,
#         y = conf_interval['upper Generated Energy'],
#         name = "Upper Bound"
#     ))

fig.update_yaxes(title="Fixed Consumed Energy [kWh]")

fig.write_html("ARIMA_figs/arimna_consumption_model1.html")

fig.show()

In [ ]:
model2 = arimamodel(training_data=train_df['Fixed Consumed'], trend='c', p=4, d=1, q=2, P=0, D=1, Q=1, m=168)
print(model2.summary())

In [ ]:
forecast2, conf_int2 = arima_forecast(data=con_df, fitted_model=model2,
                                      start_index=0, end_index=validation_df.index[-1])
forecast2

In [ ]:
y_true = forecast2['Fixed Consumed'][validation_df.index[0] : validation_df.index[-1]]
y_pred = forecast2['Prediction'][validation_df.index[0] : validation_df.index[-1]]

model2_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
print(model2_rmse)
model2_rmse

In [ ]:
fig = go.Figure(go.Scattergl(
    x = forecast2[train_df_1week : validation_df.index[0]].index,
    y = forecast2['Fixed Consumed'][train_df_1week : validation_df.index[0]],
    line=dict(color='rgb(84, 84, 84)'),
    name = "Training Data"
))

fig.add_trace(go.Scattergl(
    x = forecast2[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast2['Fixed Consumed'][validation_df.index[0] : validation_df.index[-1]],
    line=dict(color='rgb(234,143,129)'),
    name = "Validation Data"
))

fig.add_trace(go.Scattergl(
    x = forecast2[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast2['Prediction'][validation_df.index[0] : validation_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

fig.update_yaxes(title="Fixed Consumed Energy [kWh]")

fig.write_html("ARIMA_figs/arima_consumption_model2.html")

fig.show()

In [ ]:
test1 = model1.plot_diagnostics()

In [ ]:
test2 = model2.plot_diagnostics()

### SARIMAX(?, ?, ?, ?, ?, ?, 24)

In [ ]:
# model = SARIMAX(train_df_resampled['Fixed Consumed'], exog=train_df_resampled['Outdoor Temp'], trend='c',
#                  order=(1, 0, 0), seasonal_order=(1, 0, 0, 24))
# model3 = model.fit()
# print(model3.summary())

In [ ]:
# con_df['Prediction'] = model3.predict(start=0, end=validation_df.index[-1], exog=validation_df['Outdoor Temp'])
# con_df['Prediction']

In [ ]:
fig = go.Figure(go.Scattergl(
    x = train_df.index,
    y = train_df['Fixed Consumed'],
    name = "Training Data"
))

fig.add_trace(go.Scattergl(
    x = validation_df.index,
    y = validation_df['Fixed Consumed'],
    name = "Validation Data"
))

fig.add_trace(go.Scattergl(
    x = con_df[validation_df.index[0]:validation_df.index[-1]].index,
    y = con_df['Prediction'][validation_df.index[0]:validation_df.index[-1]],
    name = "Predicted Values"
))

# if bands == True:
#     fig.add_trace(go.Scattergl(
#         x = conf_interval.index,
#         y = conf_interval['lower Generated Energy'],
#         name = "Lower Bound"
#     ))
#
#     fig.add_trace(go.Scattergl(
#         x = conf_interval.index,
#         y = conf_interval['upper Generated Energy'],
#         name = "Upper Bound"
#     ))

fig.update_yaxes(title="Fixed Consumed Energy [kWh]")

#fig.show()

### Forecast test data

In [ ]:
if model1_rmse < model2_rmse:
    forecast_test, conf_int_test = arima_forecast(data=con_df, fitted_model=model1, start_index=0, end_index=con_df.index[-1])
else:
    forecast_test, conf_int_test = arima_forecast(data=con_df, fitted_model=model2, start_index=0, end_index=con_df.index[-1])

forecast_test

In [ ]:
y_true = forecast_test['Fixed Consumed'][test_df.index[0] : test_df.index[-1]]
y_pred = forecast_test['Prediction'][test_df.index[0] : test_df.index[-1]]

model_test_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
print(model_test_rmse)
model_test_rmse

In [ ]:
test_df.index[0], test_df.index[-1]

In [ ]:
fig = go.Figure(go.Scattergl(
    x = forecast_test[train_df_1week : test_df.index[0]].index,
    y = forecast_test['Fixed Consumed'][train_df_1week : test_df.index[0]],
    line=dict(color='rgb(84, 84, 84)'),
    name = "Training Data"
))

fig.add_trace(go.Scattergl(
    x = forecast_test[test_df.index[0] : test_df.index[-1]].index,
    y = forecast_test['Fixed Consumed'][test_df.index[0] : test_df.index[-1]],
    line=dict(color='rgb(234,143,129)'),
    name = "Test Data"
))

fig.add_trace(go.Scattergl(
    x = forecast_test[test_df.index[0] : test_df.index[-1]].index,
    y = forecast_test['Prediction'][test_df.index[0] : test_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

fig.update_yaxes(title="Fixed Consumed Energy [kWh]")

fig.write_html("ARIMA_figs/arima_consumption_testdata.html")

fig.show()